In [ ]:
import scannerpy 
import scannertools as st
import os
from django.db.models import Q
from query.models import Video, VideoTag, Frame
import numpy as np
from scipy.spatial import distance
from esper.prelude import load_frame
import cv2
from tqdm import tqdm

# Label Functions for Shot Detection
In this notebook we're using RGB histograms, HSV histograms, optical flow histograms, and face detections to write label functions for shot detection.

# Histograms

## Load Histograms

In [ ]:
db = scannerpy.Database()

In [ ]:
videos = Video.objects.filter(ignore_film=False).order_by('id').all()

In [ ]:
frames = [
    range(0, video.num_frames) for video in videos
]

In [ ]:
rgb_hists = st.histograms.compute_histograms(
    db,
    videos = [video.for_scannertools() for video in videos],
    frames=frames
)

In [ ]:
hsv_hists = st.histograms.compute_hsv_histograms(
    db,
    videos = [video.for_scannertools() for video in videos],
    frames=frames
)

In [ ]:
of_hists = st.histograms.compute_flow_histograms(
    db,
    videos = [video.for_scannertools() for video in videos],
    frames=frames
)

## Label functions for histograms

In [ ]:
WINDOW_SIZE = 500
POSITIVE_OUTLIER_THRESHOLD = 2.5
NEGATIVE_OUTLIER_THRESHOLD = 1.0

In [ ]:
def labels_from_color_histograms(histogram):
    histogram=list(histogram)

    # Compute the mean difference between each pair of adjacent frames
    diffs = np.array([
        np.mean([distance.chebyshev(histogram[i - 1][j], histogram[i][j]) for j in range(3)])
        for i in range(1, len(histogram))
    ])
    diffs = np.insert(diffs, 0, 0)
    n = len(diffs)

    # Do simple outlier detection to find boundaries between shots
    positive_boundaries = []
    negative_boundaries = []
    for i in range(1, n):
        window = diffs[max(i - WINDOW_SIZE, 0):min(i + WINDOW_SIZE, n)]
        if diffs[i] - np.mean(window) > POSITIVE_OUTLIER_THRESHOLD * np.std(window):
            positive_boundaries.append(i)
        if diffs[i] - np.mean(window) < NEGATIVE_OUTLIER_THRESHOLD * np.std(window):
            negative_boundaries.append(i)

    return positive_boundaries, negative_boundaries

In [ ]:
rgb_positive_labels = []
hsv_positive_labels = []
rgb_negative_labels = []
hsv_negative_labels = []
for video, rgb_hist, hsv_hist in tqdm(zip(videos, rgb_hists, hsv_hists), total=len(videos)):
    if rgb_hist is None:
        print('RGB hist for {} is None'.format(video.title))
    if hsv_hist is None:
        print('RGB hist for {} is None'.format(video.title))
    pos_rgb, neg_rgb = labels_from_color_histograms(rgb_hist.load(workers=1)) if rgb_hist is not None else [], []
    pos_hsv, neg_hsv = labels_from_color_histograms(hsv_hist.load(workers=1)) if hsv_hist is not None else [], []
    
    rgb_positive_labels.append(pos_rgb)
    hsv_positive_labels.append(pos_hsv)
    rgb_negative_labels.append(neg_rgb)
    hsv_negative_labels.append(neg_hsv)